In [5]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [6]:
%alias nbconvert nbconvert ./slimpi.ipynb
%nbconvert

[NbConvertApp] Converting notebook ./slimpi.ipynb to python


In [7]:
import logging
import logging.config

# change directory to the location where the script is running
from os import chdir

# parse arguments
import sys

# handle importing libraries based on config file
import importlib

# loop delay - sleep
from time import sleep

# clock
from datetime import datetime

##### PyPi Modules #####
# handle http requests
import requests

# rate limit the queries on the LMS server
from ratelimiter import RateLimiter

# lmsquery-fork for managing communications with lms server
import lmsquery

import constants
import epdlib
from library import configuration
from library import signalhandler
from library import cacheart

import waveshare_epd # explicitly import this to make sure that PyInstaller can find it

In [9]:
def do_exit(status=0, message=None):
    if message:
        border = '\n'+'#'*70 + '\n'
        message = border + message + border + '\n***Exiting***'
        print(message)
        
    try:
        sys.exit(status)
    except Exception as e:
        pass

In [10]:
def scan_servers():
    """scan for and list all available LMS Servers and players"""
    print(f'Scanning for available LMS Server and players')
    servers = lmsquery.scanLMS()
    if not servers:
        print('Error: no LMS servers were found on the network. Is there one running?')
        do_exit(1)
    print('servers found:')
    print(servers)
    players = lmsquery.LMSQuery().get_players()
    # print selected keys for each player
    keys = ['name', 'playerid', 'modelname']
    for p in players:
        print('players found:')
        try:
            for key in keys:
                print(f'{key}: {p[key]}')
            print('\n')
        except KeyError as e:
            pass 

In [11]:
def main():
    #### CONSTANTS ####
    # pull the absolute path from the constants file that resides in the root of the project
    absPath = constants.absPath
    # change the working directory - simplifies all the other path work later
    chdir(absPath)
    
    version = constants.version
    app_name = constants.app_name
    app_long_name = constants.app_long_name
    url = constants.url
        
    ## CONFIGURATION FILES ##
    # logging configuration file
    logging_cfg = constants.logging_cfg
    
    # default base configuration file
    default_cfg = constants.default_cfg
    system_cfg = configuration.fullPath(constants.system_cfg)
    user_cfg = configuration.fullPath(constants.user_cfg)
    
    # file for no artwork
    noartwork = constants.noartwork
    
    # set the waveshare library
    waveshare = constants.waveshare
    
    # set plugins library
    plugins = constants.plugins
    
    # file containing layouts
    layouts_file = constants.layouts
    
    default_clock = constants.clock
    
    
    # FORMATTERS
    keyError_fmt = 'KeyError: configuration file section [{}] bad/missing value: "{}"'
    configError_fmt = 'configuration file error: see section [{}] in config file: {}'
    moduleError_fmt = 'failed to load module "{}" {}'
    
    
    #### CONFIGURATION ####
    
    ## LOGGING INIT
    logging.config.fileConfig(logging_cfg)
    
    #### COMMANDLINE ARGS ####
    options = configuration.Options(sys.argv)
    # add options to the configuration object
    # options that override the configuration file options, add in the format: 
    # dest=[[ConfigFileSectionName]]__[[Option_Name]]
    #                               ^^ <-- TWO underscores `__`
    # specifying arguments with #ignore_none=True and ignore_false=True will exclude
    # these arguments entirely from the nested dictionary making it easier to merge
    # the command line arguments into the configuration file without adding unwanted options
    # with default values that potentially conflict or overwrite the config files
    
    # set logging level
    options.add_argument('-l', '--log-level', ignore_none=True, metavar='LOG_LEVEL',
                         type=str, choices=['DEBUG', 'INFO', 'WARNING', 'ERROR'], 
                         dest='main__log_level', 
                         help='set logging level: DEBUG, INFO, WARNING, ERROR')

    # alternative user_cfg file -- do not add this to the options dictionary if NONE
    options.add_argument('-c', '--config', type=str, required=False, metavar='/path/to/cfg/file.cfg',
                         dest='user_cfg', ignore_none=True, default=user_cfg,
                         help=f'use the specified configuration file; default user config: {user_cfg}')
    
    # daemon mode
    options.add_argument('-d', '--daemon', required=False,
                         default=False, dest='main__daemon', action='store_true', 
                         help='run in daemon mode (ignore user configuration)')
    
    # list servers 
    options.add_argument('-s', '--list-servers', action='store_true', 
                         dest='list_servers',
                         default=False, 
                         help='list servers and any players found on local network and exit')
    
    # set the player-id on the command line -- do not add if set to NONE
    options.add_argument('-p', '--player-name', type=str, required=False, metavar='playerName',
                         default=False, dest='lms_server__player_name', ignore_none=True,
                         help='set the name of the player to monitor')
    
    # display the version and exit
    options.add_argument('-V', '--version', action='store_true', required=False,
                         dest='version', default=False, 
                         help='display version nubmer and exit')
    
    #output the current image displayed to a temporary directory - debugging, screenshoting
    options.add_argument('-t', '--screenshot', metavar = 'INT', type=int, default=None,
                         required=False, dest='main__screenshot', ignore_none=True,
                         help='output the current screen image into the temporary folder for debugging')

    
    # parse the command line options
    options.parse_args()
    
    #### ACTION COMMAND LINE ARGUMENTS ####
    # print version and exit
    if options.options.version:
        print(f'version: {version}')
        do_exit(0)
    
    # scan for local LMS servers and players, then exit
    if options.options.list_servers:
        scan_servers()
        do_exit(0)
    
    # user a user specified configuration file
    if 'user_cfg' in options.opts_dict:
        user_cfg = options.opts_dict['user_cfg']
    
    # always try to use these two configuration files at launch
    config_file_list = [default_cfg, system_cfg]
    
    # check if running in daemon mode; append user config file
    if not options.options.main__daemon:
        config_file_list.append(user_cfg)
    
    # read all the configuration files in the list - values in left most file is default
    # values in each file to the right override previous values
    try:
        config_file = configuration.ConfigFile(config_files=config_file_list)
    except FileNotFoundError as e:
        logging.error(f'could not open one or more config files: {e}')
        logging.error('attempting to continue without above files')
        
    # merge the configuration file(s) values with command line options
    # command line options override config files
    config = configuration.merge_dict(config_file.config_dict, options.nested_opts_dict)
        
    # kludge to work around f-strings with quotes in Jupyter
    ll = config['main']['log_level']
    logging.root.setLevel(ll)
    logging.debug(f'log level set: {ll}')    
    
    #### HARDWARE INIT ####
    ## EPD INIT ##
    try:
        epd_module = '.'.join([waveshare, config['layouts']['display']])
        epd = importlib.import_module(epd_module)
#         epd = importlib.import_module('.'.join([waveshare, 'foo']))
    except KeyError as e:
        myE = keyError_fmt.format('layouts', 'display')
        logging.fatal(myE)
#         do_exit(1, message=f'Bad or missing "display" type in section [layouts] in {config_file_list}')
        do_exit(1, message=myE)

    except ModuleNotFoundError as e:
        myE = configError_fmt.format('layouts', config_file_list)
        logging.fatal(myE)
#         do_exit(1, message=f'Bad or missing "display" in section [layouts] in {config_file_list}')
        do_exit(1, message=moduleError_fmt.format(epd_module, myE))
        
    ## SCREEN INIT ##
    screen = epdlib.Screen()
    try:
        screen.epd = epd
        
    except PermissionError as e:
        logging.critical(f'Error initializing EPD interface: {e}')
        logging.critical('The user executing this program does not have access to the SPI devices.')
        do_exit(0, 'This user does not have access to the SPI group\nThis can typically be resolved by running:\n$ sudo groupadd <username> spi')
    
    screen.initEPD()
      
    
    ## LAYOUT INIT ##
    
    # import layouts
    logging.debug(f'importing layouts from file: {layouts_file}')
    try:
        layouts = importlib.import_module(layouts_file)
        playing_layout_format = getattr(layouts, config['layouts']['now_playing'])
        plugin_layout_format = getattr(layouts, config['layouts']['plugin'])
        splash_layout_format = getattr(layouts, config['layouts']['splash'])
        error_layout_format = getattr(layouts, config['layouts']['error'])
    except ModuleNotFoundError as e: 
        myE = f'- could not load layouts from module {config_file}'
        logging.fatal(moduleError_fmt.format('layouts', myE))
        do_exit(1, message=moduleError_fmt.format('layouts', myE))
    
    except (KeyError, AttributeError) as e:
        logging.fatal(keyError_fmt.format('layouts', e.args[0]))
        logging.fatal(configError_fmt.format('layotus', config_file_list))
        do_exit(1, message=keyError_fmt.format('layouts', e.args[0]))
    
    
    playing_layout = epdlib.Layout(layout=playing_layout_format, resolution=screen.resolution)
    plugin_layout = epdlib.Layout(layout=plugin_layout_format, resolution=screen.resolution)
    error_layout = epdlib.Layout(layout=error_layout_format, resolution=screen.resolution)
        
    
    ## PLUGIN INIT ##
    try:
        plugin = importlib.import_module('.'.join([plugins, config['modules']['plugin']]))
    except KeyError as e:
        logging.error(keyError_fmt.format('modules', config['modules']['plugin']))
    except ModuleNotFoundError as e:
        logging.error(moduleError_fmt.format(config['modules']['plugin'], e.args[0]))
        logging.error('falling back to default')
        try:
            plugin = importlib.import_module('.'.join([plugins, default_clock]))
            plugin_layout_format = getattr(layouts, default_clock)
        except ModuleNotFoundError as e:
            myE = moduleError_fmt.format(default_clock, e.args[0])
            logging.fatal(myE)
            do_exit(1, myE)
    
    try:
        plugin_update = int(config['modules']['plugin_update'])
    except KeyError as e:
        myE = keyError_fmt.format('modules', 'plugin_update')
        logging.error(myE)
        logging.error('setting plugin update to 60 seconds')
        pluggin_update = 60
    
    
    #### EXECUTION ####
    logging.debug(f'starting with configuration: {config}')
    
    
    
    
    ## EXEC VARIABLES ##
    # signal handler for catching and handling HUP/KILL signals
    sigHandler = signalhandler.SignalHandler()
    
    # LMS Query rate limiter wrapper - allow max of `max_calls` per `period` (seconds)
    lmsQuery_ratelimit = RateLimiter(max_calls=1, period=3)
    
    # LMS Query Object creation - rate limit to once/30 seconds
    lmsDelay_ratelimit = RateLimiter(max_calls=1, period=30)
    
    # logitech media server interface object
    lms = None
    
    # refresh when true
    refresh = False
    refresh_delay = 60
    
    # vars for managing track ID, mode, album art
    nowplaying_id = None
    nowplaying_mode = "Pause"
    artwork_cache = cacheart.CacheArt(app_long_name)
    
    if int(config['main']['screenshot']) > 0:
        store = int(config['main']['screenshot']) # f string kludge
        logging.debug(f'creating screenshot object - storing {store} images in {artwork_cache.cache_path}')
        screenshot = epdlib.ScreenShot(path=artwork_cache.cache_path, n=store)
    else:
        logging.debug('not collecting screenshots')
        screenshot = False
    
    # check for the word `true` - config file is all stored as type `str`
    if config['main']['splash_screen'].lower() == 'true':
        splash_layout = epdlib.Layout(layout=splash_layout_format, resolution=screen.resolution)
        splash_layout.update_contents({'app_name': app_name,
                                       'version': f'version: {version}',
                                       'url': url})
        refresh = splash_layout
#         screen.elements = splash_layout.blocks.values()
#         screen.concat()
#         screen.writeEPD()
    else:
        pass
        
    
    try:
        while not sigHandler.kill_now:
            response = None
        
            # check of LMS query object is configured
            if lms:
                with lmsQuery_ratelimit:
                    try:
                        myE = f'query lms server for status of player {config["lms_server"]["player_name"]}: {lms.player_id}'
                        logging.debug(myE)
                        response = lms.now_playing()
                    except requests.exceptions.ConnectionError as e:
                        logging.warning(f'server could not find active player_id: {lms.player_id}')
                        logging.warning(f'is the specified player active?')
                        logging.warning(f'error: {e}')
                        error_layout.update_contents({'message': f'{config["lms_server"]["player_name"]} does not appear to be available. Is it on?', 'time': 'NO PLAYER'})
                        refresh = error_layout
                        response = None
                        
                    except KeyError as e:
                        myE = f'No playlist is active on {config["lms_server"]["player_name"]}'
                        logging.info(myE)
                        response =  {'title': 'No music is queued', 'id': 'NONE', 'mode': 'No Playlist'}
                        nowplaying_mode = response['mode']                 
    
            else: # try to create an lms query object
                with lmsDelay_ratelimit:
                    try:
                        logging.debug('setting up lms query connection')
                        lms = lmsquery.LMSQuery(**config['lms_server'])
                        if not lms.player_id:
                            
                            raise ValueError(keyError_fmt.format('lms_server', 'player_name'))
                        logging.info('lms query connection created')

                    except TypeError as e:
                        logging.critical(f'TypeError: {e}')
                        logging.critical(configError_fmt.format('lms_server', config_file_list))
                        error_layout.update_contents({'message': configError_fmt.format('lms_server', config_file_list)})
                        refresh = error_layout
                        resp_mode = 'Error'

                    except ValueError as e:
                        myE = keyError_fmt.format('lms_server', 'player_name')
                        logging.critical(myE)
                        myE = 'LMS QUERY ERROR: \n' + myE
                        error_layout.update_contents({'message': myE, 'time': 'LMS ERROR'})
                        refresh = error_layout
                        resp_mode = 'Error'
                        
                    except OSError as e:
                        myE = 'could not find LMS servers due to network error '
                        logging.warning(myE)
                        logging.warning('delaying start of LMS query connection')
                        myE = 'LMS QUERY ERROR: ' + myE + str(e.args[0])
                        error_layout.update_contents({'message': myE, 'time': 'LMS ERROR'})
                        refresh = error_layout
                        resp_mode = 'Error'
                        

            if response:
                try:
                    resp_id = response['id']
                    resp_mode = response['mode']
                    time = response['time']
                except KeyError as e:
                    logging.error('bad response from server: e')
                    resp_id = None
                    resp_mode = 'QUERY ERROR'
                    time = 0.0001
                
                logging.debug(f'got response from server: {resp_mode}, elapsed: {time:.2f}')
                if resp_id != nowplaying_id or resp_mode != nowplaying_mode:
                    logging.info(f'track/mode change to: {resp_mode}')
                    nowplaying_id = resp_id
                    nowplaying_mode = resp_mode
                    
                    # attempt to fetch artwork 
                    try:
                        logging.debug('attempting to download artwork')
                        artwork = artwork_cache.cache_artwork(response['artwork_url'], response['album_id'])
                    except KeyError as e:
                        logging.warning('no artwork available')
                        artwork = None
                    if not artwork:
                        logging.warning(f'using default artwork file: {noartwork}')
                        artwork = noartwork
                    # add the path to the downloaded album art into the response
                    response['coverart'] = str(artwork)
                             
                    # update the layout with the values in the response
                    playing_layout.update_contents(response)
                    
                    # refresh contains the current layout
                    refresh = playing_layout
                    #set delay to 60 seconds
                    refresh_delay = 60
                else:
                    refresh = False
                
            if nowplaying_mode != "play" and screen.update.last_updated > refresh_delay:
                logging.debug(f'next update will be in {refresh_delay} seconds')
                logging.info('music appears to be paused, switching to plugin display')
                update = plugin.update()
                update['mode'] = nowplaying_mode
                plugin_layout.update_contents(update)
                refresh = plugin_layout
                refresh_delay = plugin_update                
            
            
            # check if `refresh` has been updated 
            if refresh and isinstance(refresh, epdlib.Layout):
                logging.debug('refresh display')
                screen.initEPD()
                image = refresh.concat()
#                 screen.elements = refresh.blocks.values()
#                 image = screen.concat()
                screen.writeEPD(image)
                
                if screenshot:
                    screenshot.save(image)
                
                refresh = False
            else:
                logging.warning(f'refresh called, but type: {type(refresh)}; skipping')
            # sleep for half a second every cycle
            sleep(0.5)
            
    finally:
        print('Received exit signal - cleaning up')
        
        screen.initEPD()
        screen.clearEPD()
        artwork_cache.clear_cache()
        
    return config

In [7]:
if __name__ == '__main__':
    o = main()

18:05:30 <ipython-input-6-af9cbe5ecca5>:main:137:DEBUG - log level set: DEBUG
18:05:30 Screen:__init__:228:INFO - Screen created
18:05:30 epd5in83:ReadBusy:69:DEBUG - e-Paper busy
18:05:31 epd5in83:ReadBusy:72:DEBUG - e-Paper busy release
18:05:31 Screen:initEPD:310:INFO - <waveshare_epd.epd5in83.EPD object at 0xb40a3830> initialized
18:05:31 <ipython-input-6-af9cbe5ecca5>:main:173:DEBUG - importing layouts from file: layouts
18:05:31 Layout:__init__:170:DEBUG - creating layout
18:05:31 Layout:layout:205:DEBUG - calculating layout for resolution [600, 448]
18:05:31 Layout:layout:211:DEBUG - layout id(2957024304)
18:05:31 Layout:_calculate_layout:234:DEBUG - *****title*****
18:05:31 Layout:_check_keys:383:DEBUG - checking layout keys
18:05:31 Layout:_check_keys:388:DEBUG - adding key: rand: False
18:05:31 Layout:_check_keys:388:DEBUG - adding key: inverse: False
18:05:31 Layout:_check_keys:388:DEBUG - adding key: maxchar: None
18:05:31 Layout:_check_keys:388:DEBUG - adding key: dimensio

18:05:31 Block:_text_formatter:493:DEBUG - formatting string: .
18:05:31 Block:_text_formatter:496:DEBUG - formatted list:
 ['.']
18:05:31 Block:_text2image:507:DEBUG - creating blank image area: (360, 112) with inverse: False
18:05:31 Block:_text2image:522:DEBUG - line size: 8, 41
18:05:31 Block:_text2image:526:DEBUG - max x dim so far: 8
18:05:31 Block:_text2image:530:DEBUG - dimensions of text portion of image: (8, 41)
18:05:31 Block:_text2image:540:DEBUG - drawing text at 0, 0
18:05:31 Block:_text2image:541:DEBUG - with dimensions: 8, 41
18:05:31 Block:_text2image:567:DEBUG - pasting text portion at coordinates: 0, 36
18:05:31 Layout:_set_images:346:DEBUG - ***album***)
18:05:31 Layout:_set_images:350:DEBUG - set text block
18:05:31 Block:__init__:344:INFO - TextBlock created
18:05:31 Block:maxchar:433:DEBUG - no maxchar set
18:05:31 Block:font:403:DEBUG - setting old_font = None
18:05:32 Block:_calc_maxchar:457:DEBUG - calculating maximum characters for font ('Anton', 'Regular')
1

18:05:33 Block:_text2image:522:DEBUG - line size: 10, 48
18:05:33 Block:_text2image:526:DEBUG - max x dim so far: 10
18:05:33 Block:_text2image:530:DEBUG - dimensions of text portion of image: (10, 48)
18:05:33 Block:_text2image:540:DEBUG - drawing text at 0, 0
18:05:33 Block:_text2image:541:DEBUG - with dimensions: 10, 48
18:05:33 Block:_text2image:551:DEBUG - randomly positioning text within area
18:05:33 Block:_text2image:567:DEBUG - pasting text portion at coordinates: 131, 3
18:05:33 Layout:_set_images:346:DEBUG - ***mode***)
18:05:33 Layout:_set_images:350:DEBUG - set text block
18:05:33 Block:__init__:344:INFO - TextBlock created
18:05:33 Block:maxchar:433:DEBUG - no maxchar set
18:05:33 Block:font:403:DEBUG - setting old_font = None
18:05:33 Block:_calc_maxchar:457:DEBUG - calculating maximum characters for font ('Anton', 'Regular')
18:05:33 Block:_calc_maxchar:471:DEBUG - maximum characters per line: 16
18:05:33 Block:_text_formatter:493:DEBUG - formatting string: .
18:05:33 B

18:05:34 Layout:_scalefont:297:DEBUG - calculating maximum font size for area: (600, 269)
18:05:34 Layout:_scalefont:298:DEBUG - using font: /home/pi/src/slimpi_epd/fonts/Anton/Anton-Regular.ttf
18:05:34 Layout:_scalefont:307:DEBUG - target X font dimension 705.8823529411765
18:05:34 Layout:_scalefont:308:DEBUG - target Y font dimension 201.75
18:05:34 Layout:_scalefont:327:DEBUG - Y target reached
18:05:34 Layout:_scalefont:331:DEBUG - test string: W W W ; pixel dimensions for fontsize 176: (501, 202)
18:05:34 Layout:_calculate_layout:234:DEBUG - *****version*****
18:05:34 Layout:_check_keys:383:DEBUG - checking layout keys
18:05:34 Layout:_check_keys:388:DEBUG - adding key: dimensions: None
18:05:34 Layout:_check_keys:388:DEBUG - adding key: scale_x: None
18:05:34 Layout:_check_keys:388:DEBUG - adding key: scale_y: None
18:05:34 Layout:_calculate_layout:240:DEBUG - dimensions: (600, 45)
18:05:34 Layout:_calculate_layout:249:DEBUG - section has calculated position
18:05:34 Layout:_cal

18:05:35 Block:_text2image:540:DEBUG - drawing text at 0, 0
18:05:35 Block:_text2image:541:DEBUG - with dimensions: 532, 61
18:05:35 Block:_text2image:538:DEBUG - hcenter line: limpi_epd
18:05:35 Block:_text2image:540:DEBUG - drawing text at 176, 61
18:05:35 Block:_text2image:541:DEBUG - with dimensions: 179, 61
18:05:35 Block:_text2image:567:DEBUG - pasting text portion at coordinates: 34, 6
18:05:35 <ipython-input-6-af9cbe5ecca5>:main:299:DEBUG - setting up lms query connection
18:05:35 <ipython-input-6-af9cbe5ecca5>:main:307:CRITICAL - TypeError: an integer is required (got type LMSQuery)
18:05:35 <ipython-input-6-af9cbe5ecca5>:main:308:CRITICAL - configuration file error: see section [lms_server] in config file: ['./slimpi.cfg', PosixPath('/etc/slimpi.cfg'), PosixPath('/home/pi/.config/com.txoof.slimpi/slimpi.cfg')]
18:05:35 Layout:update_contents:400:INFO - updating blocks
18:05:35 Layout:update_contents:406:DEBUG - updating block: message
18:05:35 Block:_text_formatter:493:DEBUG 

18:06:33 cacheart:clear_cache:91:DEBUG - clearing previously downloaded files in /tmp/com.txoof.slimpi


In [ ]:
dir(lmsquery)

In [ ]:
import lmsquery